In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame
    
Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = "census_data.csv"

weather_df = pd.read_csv(csv_file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,47.68,76,96,1.01,US,1635037565
1,Lanigan,51.8501,-105.0344,41.65,54,100,16.53,CA,1635037883
2,Mataura,-46.1927,168.8643,64.89,61,100,1.07,NZ,1635037710
3,Yellowknife,62.4560,-114.3525,41.56,86,100,13.00,CA,1635037495
4,Pahrump,36.2083,-115.9839,73.06,36,1,1.99,US,1635037883


# Humidity Heatmap

Configure gmaps.

Use the Lat and Lng as locations and Humidity as the weight.

Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Drop Nan values
weather_df = weather_df.dropna()

#define Lat and Lng as locations
locations = weather_df[['Lat', 'Lng']].astype(float)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

#set humidity as the weight
weights = weather_df['Humidity'].astype(float)

In [5]:
#create map
fig = gmaps.figure()


#add heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.

In [9]:
#New Data Frame with ideal travel conditions (Max Temp < 90, Max Temp > 75, Humidity < 60, Wind Speed < 8, clouds = 0)

ideal_weather_df = weather_df.loc[(weather_df['Max Temp'] < 90) & (weather_df['Max Temp'] > 75) & (weather_df['Wind Speed'] < 8) & (weather_df['Humidity'] < 60) & (weather_df['Clouds'] == 0)]
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
39,Geraldton,-28.7667,114.6000,76.12,56,0,1.14,AU,1635037845
286,Sahuaripa,29.0500,-109.2333,75.43,33,0,6.17,MX,1635037946
413,Agadez,19.7500,10.2500,77.47,17,0,2.51,NE,1635037782
481,Arlit,18.7369,7.3853,83.28,15,0,7.20,NE,1635037964
530,Atar,20.5169,-13.0499,78.94,18,0,7.70,MR,1635038006


# Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [19]:
#Create a new Data Frame
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

#Set parameters to search for hotels within 5000 meters
params = {"radius": 5000, "types" : "hotel", "keyword" : "hotels", "key" : g_key}

# Loop through the lat and lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(base_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
hotel_df.head()

{
    "html_attributions": [],
    "next_page_token": "Aap_uECID8DsxXkutrTL7mCYG99ptUs23g2tLOwXBXNgsl-Db0t5uj59-y-5lqEenIgaTmwlvB0tFIVdjxMZwPIs3Svts6hEP9YDveij0zmi0VRIYf6_rs9yCHvwCOz0AJvYLgURwNvtG7QYQlHK9TXkWOLDihnrmvayYRwO9wj4YKZomHDbP22WdX_eL1AQIMZ1sO2MoQOVoTuvMSy3EvBxlySN4-p5BGPpb_EJxRPQkBa-ai0ewaSGcPegGqR-dpyCvJp1xSwk2BwJHcWccQYZ--WYB2oyiIizBEBn_Hc4dQTLmoNs_Hat39mrKnbwiixLBsFB-i49MMkIAgglfChLWHY__Jd1jmfUcNv5ZS1skcvoO_1qPk6xB87JqY3l-ZSNDWacRq1w1fHJ4BLMWCuRTX7pzbPF5sZhFymEKGiFqnHNESwle7BqTqwsZABm7IFq",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.7738192,
                    "lng": 114.6082717
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.77237427010728,
                        "lng": 114.6095486298927
                    },
                    "southwest": {
                        "lat": -28.77507

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 29.0498524,
                    "lng": -109.2320328
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.05121112989272,
                        "lng": -109.2306079701073
                    },
                    "southwest": {
                        "lat": 29.04851147010728,
                        "lng": -109.2333076298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel El Molino de Don Pepe",
            "photos": [
                {
                    "h

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.519051,
                    "lng": -13.0621033
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.52039992989272,
                        "lng": -13.06077727010728
                    },
                    "southwest": {
                        "lat": 20.51770027010728,
                        "lng": -13.06347692989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Bab Sahara",
            "photos": [
                {
                    "height": 3024,
     

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
0,Geraldton,-28.7667,114.6000,76.12,56,0,1.14,AU,1635037845,Ocean Centre Hotel
1,Sahuaripa,29.0500,-109.2333,75.43,33,0,6.17,MX,1635037946,Hotel El Molino de Don Pepe
2,Agadez,19.7500,10.2500,77.47,17,0,2.51,NE,1635037782,
3,Arlit,18.7369,7.3853,83.28,15,0,7.20,NE,1635037964,Hôtel Temet
4,Atar,20.5169,-13.0499,78.94,18,0,7.70,MR,1635038006,Bab Sahara


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))